In [1]:
import numpy as np

In [2]:
def hello():
    nodes = 1

hello()

array([4.65822458e-310, 0.00000000e+000, 6.95093654e-310, 6.95093715e-310,
       6.95093654e-310, 6.95093663e-310, 6.95093713e-310, 6.95093654e-310,
       6.95093654e-310, 4.65932845e-310, 6.95093715e-310, 6.95093715e-310,
       6.95093715e-310, 6.95093654e-310, 6.95093654e-310, 6.95093654e-310,
       6.95093654e-310, 6.95093713e-310, 6.95093664e-310, 6.95093654e-310,
       6.95093654e-310, 6.95093654e-310, 6.95093663e-310, 6.95093654e-310,
       6.95093714e-310, 6.95093654e-310, 6.95093661e-310, 6.95093662e-310,
       6.95093665e-310, 6.95093663e-310, 6.95093715e-310, 6.95093654e-310,
       6.95093654e-310, 6.95093714e-310, 6.95093665e-310, 6.95093714e-310,
       6.95093654e-310, 6.95093654e-310, 6.95093661e-310, 6.95093715e-310,
       6.95093654e-310, 6.95093654e-310, 6.95093654e-310, 6.95093654e-310,
       6.95093661e-310, 6.95093661e-310, 6.95093663e-310, 6.95093654e-310,
       6.95093654e-310, 6.95093662e-310])